In [64]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint
from itertools import combinations

# Atsisiunciame duomenis ( pvz . NVDA ir BTC )
tickers = ["BTC-USD", "BNB-USD", 'KC=F', 'SB=F', 'DX-Y.NYB', 'GC=F']
df = yf.download(tickers, start="2016-01-01")
df = df.ffill().bfill() # Uzpildome praleistas reiksmes, kadangi dalis musu portfelio yra derivatyvai
returns = df["Close"].pct_change().dropna()


[*********************100%***********************]  6 of 6 completed


In [52]:
print(returns)
print()
print(returns.mean())

Ticker       BTC-USD  DOGE-USD  DX-Y.NYB      GC=F      KC=F      SB=F
Date                                                                  
2018-01-02  0.097011  0.026490  0.000000  0.000000  0.000000  0.000000
2018-01-03  0.014611  0.019136  0.003375  0.001903 -0.012289 -0.001305
2018-01-04  0.026196  0.034764 -0.003364  0.002431  0.007387 -0.003919
2018-01-05  0.117333  0.261613  0.001089  0.000682 -0.008491 -0.011148
2018-01-06  0.005594  0.221583  0.000000  0.000000  0.000000  0.000000
...              ...       ...       ...       ...       ...       ...
2026-02-22 -0.005064 -0.030717  0.000000  0.000000  0.000000  0.000000
2026-02-23 -0.044970 -0.030172 -0.001023  0.028739 -0.024801  0.010489
2026-02-24 -0.008306 -0.012001  0.001842 -0.009395  0.025253  0.006920
2026-02-25  0.060551  0.101430 -0.001839  0.009814  0.000173  0.002749
2026-02-26  0.002898 -0.022063 -0.000041 -0.002939 -0.020985 -0.037697

[2978 rows x 6 columns]

Ticker
BTC-USD     0.001121
DOGE-USD    0.003379
DX

In [3]:
import plotly.io as pio
pio.renderers.default = "notebook" 

In [4]:
coint( df['Close']['KC=F'], df['Close']['SB=F'] )

(np.float64(-1.2101016680194274),
 np.float64(0.8552359335498396),
 array([-3.90012138, -3.33818252, -3.04587448]))

In [60]:
risk_free_rate = 0.03  # Tarkime , JAV obligaciju palukanos 3%


def analyze_asset(df, weights: list[float]):
    """
    Kainos grafiką su indikatoriumi. - Candlestick
    95% VaR reikšmę.
    Expected Shortfall
    Sharpe koeficientą.
    """

    assert np.isclose( sum(weights), 1.0), "Svoriai turi susisumuoti i 1"
    assert len(weights) == df['Close'].shape[1],\
        f"Svoriai turi atitikti instrumentus, {len(weights)}, {df.shape[1]}"

    print("Viso portfelio analizė")
    print("-"*20)

    # Apskaiciuoti bendras portfelio vertes
    returns = df['Close']\
        .apply( lambda x: np.dot(x, weights), axis=1)\
        .pct_change()\
        .dropna()

    returns_individual = df['Close'].pct_change().dropna()

    print(np.round(np.corrcoef(returns_individual.T), 4))

    print( "Vidutinė grąža:", round(returns.mean(), 6) )

    annual_return = returns.mean() * 365
    annual_std = returns.std() * np.sqrt(365)

    sharpe_ratio = (annual_return - risk_free_rate) / annual_std
    print(f"Šarpo rodiklis: { sharpe_ratio :.2f} ")

    # Apskaiciuoti VaR atskiriems 
    var = np.percentile ( returns , 5)
    print( f'Istorinis VaR 95 %: {var:.2f}' )

    # Apskaiciuoti ES
    es = returns [ returns <= var ].mean()
    print( f'Expected Shortfall ( ES ) 95 %: {es:.2f}' )

    cointegrated_pairs = []

    # Iterate over all unique pairs of columns
    close_prices = df['Close']
    for col1, col2 in combinations(close_prices.columns, 2):
        score, p_value, _ = coint(close_prices[col1], close_prices[col2])
        if p_value < 0.05:
            cointegrated_pairs.append((col1, col2))
            print(f"Cointegrated pair: {col1} & {col2} (p-value={p_value:.4f})")
    
    if not cointegrated_pairs:
        print("No cointegrated pairs found.")


    # Analizuoti kiekviena atskirai
    for ticker in tickers:
        print('\n', ticker, 'analizė')
        print( '-' * 20 )

        df_plot = df

        if isinstance(df_plot.columns, pd.MultiIndex):
            df_plot = df_plot.xs(ticker, axis=1, level=1)

        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=df_plot.index,
                    open=df_plot["Open"],
                    high=df_plot["High"],
                    low=df_plot["Low"],
                    close=df_plot["Close"],
                    name="Market Data",
                )
            ]
        )
        fig.update_layout(
            title=f"{ticker} interaktyvus grafikas",
            yaxis_title="Kaina",
            xaxis_title="Data",
        )
        fig.show()

        returns = df["Close"][ticker].pct_change().dropna()
        print( "Vidutinė grąža:", returns.mean())

        if "-USD" in ticker: # Jeigu kriptovaliuta, tuomet prekyba vyksta visus metus
            annual_return = returns.mean() * 365
            annual_std = returns.std() * np.sqrt(365)
        else:
            annual_return = returns.mean() * 252
            annual_std = returns.std() * np.sqrt(252)

        sharpe_ratio = (annual_return - risk_free_rate) / annual_std
        print(f"Šarpo rodiklis: { sharpe_ratio :.2f} ")

        # Apskaiciuoti VaR atskiriems 
        var = np.percentile ( returns , 5)
        print( f'Istorinis VaR 95 %: {var:.2f}' )

        # Apskaiciuoti ES
        es = returns [ returns <= var ].mean()
        print( f'Expected Shortfall ( ES ) 95 %: {es:.2f}' )

In [65]:
analyze_asset(df, [0.2, 0.1, 0.2, 0.2, 0.05, 0.25])

Viso portfelio analizė
--------------------
[[ 1.      0.5658 -0.0662  0.0644  0.0282  0.0647]
 [ 0.5658  1.     -0.0762  0.0806  0.0425  0.0448]
 [-0.0662 -0.0762  1.     -0.4028 -0.1077 -0.0524]
 [ 0.0644  0.0806 -0.4028  1.      0.1055  0.0932]
 [ 0.0282  0.0425 -0.1077  0.1055  1.      0.1868]
 [ 0.0647  0.0448 -0.0524  0.0932  0.1868  1.    ]]
Vidutinė grąža: 0.001261
Šarpo rodiklis: 0.84 
Istorinis VaR 95 %: -0.04
Expected Shortfall ( ES ) 95 %: -0.06
Cointegrated pair: BNB-USD & BTC-USD (p-value=0.0309)
Cointegrated pair: BNB-USD & KC=F (p-value=0.0094)
Cointegrated pair: BTC-USD & KC=F (p-value=0.0276)

 BTC-USD analizė
--------------------


Vidutinė grąža: 0.0019848564106818593
Šarpo rodiklis: 1.03 
Istorinis VaR 95 %: -0.05
Expected Shortfall ( ES ) 95 %: -0.08

 BNB-USD analizė
--------------------


Vidutinė grąža: 0.002554220906393938
Šarpo rodiklis: 1.03 
Istorinis VaR 95 %: -0.06
Expected Shortfall ( ES ) 95 %: -0.10

 KC=F analizė
--------------------


Vidutinė grąža: 0.00037471304407148977
Šarpo rodiklis: 0.23 
Istorinis VaR 95 %: -0.03
Expected Shortfall ( ES ) 95 %: -0.04

 SB=F analizė
--------------------


Vidutinė grąža: 9.156113241238039e-05
Šarpo rodiklis: -0.03 
Istorinis VaR 95 %: -0.02
Expected Shortfall ( ES ) 95 %: -0.03

 DX-Y.NYB analizė
--------------------


Vidutinė grąža: 2.9365610665485546e-06
Šarpo rodiklis: -0.53 
Istorinis VaR 95 %: -0.01
Expected Shortfall ( ES ) 95 %: -0.01

 GC=F analizė
--------------------


Vidutinė grąža: 0.000460107694177779
Šarpo rodiklis: 0.64 
Istorinis VaR 95 %: -0.01
Expected Shortfall ( ES ) 95 %: -0.02
